# Calculating Perplexity

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
#input_ids = tokenizer(prompt, return_tensors="pt").input_ids

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
import torch
prompts = [ "Words hold immense power. They can inspire, uplift, and transform lives. A well-crafted sentence has the ability to ignite imagination, stir emotions, and leave an indelible mark on the human psyche. It's a testament to the remarkable capacity of language to transcend barriers and forge connections.",
    "The natural world is a tapestry of wonders. From the majestic peaks of snow-capped mountains to the intricate dance of life in the depths of the oceans, nature's artistry is a constant source of awe and reverence. It reminds us of our humble place in the grand scheme of existence and our responsibility to protect its fragile beauty.",
    "Creativity is the lifeblood of progress. It fuels innovation, drives artistic expression, and propels us forward. In a world that often values conformity, embracing our creative spirit allows us to challenge the status quo, question the established norms, and pave the way for new possibilities.",
    "Resilience is the hallmark of the human spirit. It is the ability to rise above adversity, to adapt and evolve in the face of challenges. Like a sturdy oak that withstands the fiercest storms, resilience empowers us to bend but never break, emerging stronger and wiser from life's trials.",
    "Kindness is a universal language that transcends boundaries and cultures. A simple act of compassion, a gentle word of encouragement, or a genuine smile can have a profound impact, creating ripples of positivity that touch the lives of others. In a world that often appears harsh and unforgiving, kindness is a beacon of hope and humanity."]
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): La

In [ ]:

torch.random.manual_seed(42)
#Top- K sampling
generation_params = {
    #"greedy_search": {"do_sample": False, "max_length": 50},
    #"beam_search": {"num_beams": 5, "max_length": 50, "early_stopping": True},
    #"top_k_sampling": {"do_sample": True, "top_k": 50, "max_length": 50},
    "top_p_sampling": {"do_sample": True, "top_p": 0.6, "max_length": 30, "min_length": 0 }
}

In [ ]:
def calculate_perplexity(generated_text):
    encodings = tokenizer(generated_text, return_tensors="pt").to(device)
    input_ids = encodings.input_ids
    stride = 512
    max_length = 1024
    seq_len = input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    for begin_loc in range(0, seq_len, stride):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids_segment = input_ids[:, begin_loc:end_loc]
        target_ids = input_ids_segment.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids_segment, labels=target_ids)
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break
    return torch.exp(torch.stack(nlls).mean())

In [ ]:
perplexities = {}
for prompt in prompts:
  for strategy_name, params in generation_params.items():
      input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
      generated_outputs = model.generate(input_ids, **params, pad_token_id=tokenizer.eos_token_id)
      generated_text = tokenizer.decode(generated_outputs[0], skip_special_tokens=True)
      perplexity = calculate_perplexity(generated_text)
      perplexities[strategy_name] = perplexity
      print(f"Strategy: {strategy_name}\nGenerated Text: {generated_text}\nPerplexity: {perplexity}\n")

Strategy: top_p_sampling
Generated Text: A well-crafted sentence has the ability to ignite imagination, stir emotions, and leave an indelible mark on the human
Perplexity: 1.7641046047210693

Strategy: top_p_sampling
Generated Text: The natural world is a tapestry of wonders. From the majestic peaks of snow-capped mountains to the intricate dance of life
Perplexity: 1.5699129104614258

Strategy: top_p_sampling
Generated Text: Creativity is the lifeblood of progress. It fuels innovation, drives artistic expression, and propels us forward. In a world that
Perplexity: 1.4054306745529175

Strategy: top_p_sampling
Generated Text: Resilience is the hallmark of the human spirit. It is the ability to rise above adversity. Like a sturdy oak that withstands
Perplexity: 1.5952272415161133

Strategy: top_p_sampling
Generated Text: Kindness is a universal language that transcends boundaries and cultures. A simple act of compassion, a gentle word of encouragement, or a
Perplexity: 1.4771292209625244

# LOADING DATASET

In [ ]:
from datasets import load_dataset
cnn=load_dataset("cnn_dailymail", '2.0.0')

In [ ]:
cnn['train']['article'][0]

'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details o

In [ ]:
#def preprocess_function(examples):
    #return tokenizer(examples['article'], truncation=True)

In [ ]:
#tokenized_cnn = cnn.map(preprocess_function, batched=True)

In [ ]:
#article_input_ids = tokenized_cnn['test']['input_ids'][0]

In [ ]:
#input_tensors = [torch.tensor([ids]) for ids in article_input_ids]

In [ ]:
#outputs = []
#for input_tensor in input_tensors:
#    output = model.generate(input_tensor, **params, pad_token_id=tokenizer.eos_token_id)
 #   tokenizer.batch_decode(output, skip_special_tokens=True)

In [ ]:
#output = model.generate(article_input_ids, **params, pad_token_id=tokenizer.eos_token_id)
#tokenizer.batch_decode(output, skip_special_tokens=True)

In [ ]:
# Counter to track generated summaries
#summary_count = 0

#for article in cnn['test']['article']:
  # Break the loop if 50 summaries are generated
 # if summary_count >= 50:
 #   break

 # tokenized_article = tokenizer(article, truncation=True)
 # input_ids = torch.tensor(tokenized_article['input_ids'])
 # output = model.generate(input_ids, **params, pad_token_id=tokenizer.eos_token_id)

  # Process the generated summary (e.g., print or store)
  #print(f"Summary {summary_count + 1}:", output[0]['generated_text'])  # Accessing generated text from the first output element

 # summary_count += 1


In [ ]:
cnn['test']['article'][0]

'(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony

In [ ]:
import pandas as pd
df=pd.DataFrame()

In [ ]:
#!pip install evaluate, rouge_score, bert_score

# SETTING UP EVALUATION METRICS

In [ ]:
import evaluate
rouge_score=evaluate.load('rouge')
bert_score=evaluate.load('bertscore')

# Loop Through first 50 items to generate summaries

In [ ]:
import pandas as pd

rouge_1_scores = []
rouge_2_scores = []
rouge_l_scores = []
rouge_lsum_scores = []
bert_precision_scores = []
bert_recall_scores = []
bert_f1_scores = []

for i in range(50):
    a = cnn['test']['article'][i]
    r=cnn[f'test']['highlights'][i]
    b = tokenizer(a, truncation=True, return_tensors="pt", max_length=1024).input_ids.to(device)
    o = model.generate(b, **params, pad_token_id=tokenizer.eos_token_id)
    gt = tokenizer.decode(o[0], skip_special_tokens=True)

    rouge_score_value = rouge_score.compute(predictions=[gt], references=[cnn['test']['highlights'][i]])
    #print(rouge_score_value.keys())
    rouge_1_scores.append(rouge_score_value['rouge1'])
    rouge_2_scores.append(rouge_score_value['rouge2'])
    rouge_l_scores.append(rouge_score_value['rougeL'])
    rouge_lsum_scores.append(rouge_score_value['rougeLsum'])

    bert_score_value = bert_score.compute(predictions=[gt], references=[cnn['test']['highlights'][i]], lang="en")
    #print(bert_score_value.keys())
    bert_precision_scores.append(bert_score_value['f1'][0])
    bert_recall_scores.append(bert_score_value['recall'][0])
    bert_f1_scores.append(bert_score_value['f1'][0])

    # Create a new DataFrame or append to an existing one
    if i == 0:
        df = pd.DataFrame({'Input Text': [a], 'Generated Text': [gt], 'Rouge Score': [rouge_score_value], 'BERT Score': [bert_score_value]})
    else:
        new_row = pd.DataFrame({'Input Text': [a], 'Generated Text': [gt], 'Rouge Score': [rouge_score_value], 'BERT Score': [bert_score_value]}, index=[i])
        df = pd.concat([df, new_row], ignore_index=True)

    print(f"Generated text {i}: {gt}\n Rouge: {rouge_score_value} \n Bert_Score: {bert_score_value}")

Generated text 0: The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories.
 Rouge: {'rouge1': 0.44827586206896547, 'rouge2': 0.25, 'rougeL': 0.31034482758620696, 'rougeLsum': 0.44827586206896547} 
 Bert_Score: {'precision': [0.9076894521713257], 'recall': [0.8876417279243469], 'f1': [0.8975536227226257], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.38.2)'}
Generated text 1: A dog apparently hit by a car and buried in a field survives. The dog, now named Theia, was found emaciated
 Rouge: {'rouge1': 0.4375, 'rouge2': 0.25806451612903225, 'rougeL': 0.40625, 'rougeLsum': 0.40625} 
 Bert_Score: {'precision': [0.9049350619316101], 'recall': [0.8699348568916321], 'f1': [0.8870898485183716], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.38.2)'}
Generated text 2: Mohammad Javad Zarif is the Iranian foreign minister. He has been John Kerry's

In [ ]:
avg_rouge_1 = sum(rouge_1_scores) / len(rouge_1_scores)
avg_rouge_2 = sum(rouge_2_scores) / len(rouge_2_scores)
avg_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
avg_rouge_lsum = sum(rouge_lsum_scores) / len(rouge_lsum_scores)
avg_bert_precision = sum(bert_precision_scores) / len(bert_precision_scores)
avg_bert_recall = sum(bert_recall_scores) / len(bert_recall_scores)
avg_bert_f1 = sum(bert_f1_scores) / len(bert_f1_scores)

In [ ]:
print(f"Average ROUGE-1 F1: {avg_rouge_1}")
print(f"Average ROUGE-2 F1: {avg_rouge_2}")
print(f"Average ROUGE-L F1: {avg_rouge_l}")
print(f"Average ROUGE-LSum F1: {avg_rouge_lsum}")
print(f"Average BERT Precision: {avg_bert_precision}")
print(f"Average BERT Recall: {avg_bert_recall}")
print(f"Average BERT F1: {avg_bert_f1}")

Average ROUGE-1 F1: 0.34656756860018945
Average ROUGE-2 F1: 0.17562301578724493
Average ROUGE-L F1: 0.2866074194233454
Average ROUGE-LSum F1: 0.31418048373863267
Average BERT Precision: 0.8752924418449402
Average BERT Recall: 0.8638383138179779
Average BERT F1: 0.8752924418449402


# Export Results

In [ ]:
df.to_csv('Top-P Sampling.csv')

In [ ]:
from google.colab import files
files.download('Top-P Sampling.csv')

# Coherence


In [ ]:
from nltk import agreement
rater1=[5,5,4,5,4,3,5,4,3,4,4,5,4,3,3,4,5,4,4,4]
rater2=[5,5,4,5,4,4,5,4,3,4,4,5,4,3,3,4,5,4,4,4]
rater3=[5,5,4,5,4,5,5,4,3,4,4,5,4,4,3,4,5,4,4,4]
rater4=[5,5,5,3,5,4,5,4,3,4,4,3,4,4,3,5,4,4,5,5]

# Factuality

In [ ]:
rater1 = [4,4,3,4,5,4,5,4,5,5,5,4,5,4,4,5,4,5,5,4]
rater2 = [4,3,4,5,5,4,5,4,5,5,5,5,5,4,4,5,5,5,5,5]
rater3 = [5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5]
rater4 = [4,4,5,3,4,5,3,4,2,3,4,3,4,3,2,5,3,4,5,5]

# Formality

In [ ]:
rater1 = [3,4,4,4,3,4,4,3,4,4,4,4,4,3,4,3,3,4,4,5]
rater2 = [3,4,4,5,4,3,4,4,5,4,4,4,3,4,4,4,3,4,4,5]
rater3 = [5,4,4,5,4,4,4,4,4,4,4,5,4,4,3,4,5,4,4,4]
rater4 = [4,4,4,3,5,4,4,4,3,4,3,3,3,3,2,5,5,3,5,4]

In [ ]:
taskdata = [[0, str(i), str(rater1[i])] for i in range(0, len(rater1))] + \
           [[1, str(i), str(rater2[i])] for i in range(0, len(rater2))] + \
           [[2, str(i), str(rater3[i])] for i in range(0, len(rater3))] + \
           [[3, str(i), str(rater4[i])] for i in range(0, len(rater4))]
ratingtask = agreement.AnnotationTask(data=taskdata)
print("alpha " + str(ratingtask.alpha()))

alpha 0.015297450424929027
